<a href="https://colab.research.google.com/github/Karalius/etsy-scraper/blob/master/scraping_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies and hide cell output
!pip install fake-useragent &> /dev/null
# pyopenssl helps to avoid timeouts/bans/recaptcha
!pip install pyopenssl &> /dev/null
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from fake_useragent import UserAgent
from sqlalchemy import create_engine
from lxml.html import fromstring
from google.colab import files
from bs4 import BeautifulSoup
from typing import Set
import pandas as pd
import numpy as np
import psycopg2
import requests
import math
import time

Heroku Postgres credentials, define here:

In [ ]:
DATABASE = ""
USER = ""
PASSWORD = ""
HOST = ""
PORT = ""
sqlalchemy_engine_url = ''

In [ ]:
def get_proxies() -> Set:
    """
    This function fetches html from the www.us-proxy.org website and iterates through it.
    While iterating it collects IP addresses and ports, then joins them to form a string.
    :return: a set of strings containing proxies based in the US."""

    url = "https://www.us-proxy.org/"
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()

    # Extract proxy string and add to the set
    for i in parser.xpath("//tbody/tr")[:60]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join(
                [i.xpath(".//td[1]/text()")[0], i.xpath(".//td[2]/text()")[0]]
            )
            proxies.add(proxy)

    return proxies

In [ ]:
def replace_attr(
    html_doc: requests.models.Response, from_attr: str, to_attr: str
) -> BeautifulSoup:
    """
    Function takes the arguments explained below and returns edited html page with new tags
    :param html_doc: takes a response page, for example: requests.get(url)
    :param from_attr: give a tag name which meant to be changed, for example: 'data-src'
    :param to_attr: give a tag name which should replace from_attr, for example: 'src'
    :return: returns response page with new tag name in the html
    """

    soup = BeautifulSoup(html_doc.content, "html.parser")
    tags = soup(attrs={from_attr: True})

    # Replace tags with new tag
    for tag in tags:
        tag[to_attr] = tag[from_attr]
        del tag[from_attr]

    return soup

Available keywords:

prints, drawing-and-illustration, painting, collectibles, photography, sculpture, dolls-and-miniatures, fiber-arts, fine-art-ceramics, mixed-media-and-college, glass-art, artist-tradings-cards

In [ ]:
def scrape_etsy(keywords: list, items_to_scrape: int) -> pd.DataFrame:
    """
    This function scrapes https://www.etsy.com/c/art-and-collectibles for the given number of items for each of keyword.
    It features rotating user-agents mirroring a real agent, also rotating US proxies to avoid suspicion (bans).
    This function returns pandas dataframe with the collected information:
    Category (keyword), title, price, item url, image url.
    :param keywords: list of keywords to scrape.
    Available keywords: painting, photography, prints.
    Constrain: up to 12 keywords in the list! Check the website for the names.
    Example, keyword on the web: Drawing & Illustration -> ['drawing-and-illustration'] for the scraper.
    :param items_to_scrape: integer of items to scrape for each keyword.
    For example: 3000 - every keyword in the keywords list will be scraped for 3000 items.
    :return: pandas dataframe with the following columns:
    Category (keyword), title, price, item url, image url.
    """
    average_items_per_page = 40
    pages_to_scrape = math.ceil(items_to_scrape / average_items_per_page)
    df_list = []

    ua = UserAgent(use_cache_server=False)

    for key in keywords:
        titles, prices, item_urls, img_urls = ([] for i in range(4))
        adj_keyword = key.lower()
        proxies_set = get_proxies()

        for page_no in range(1, pages_to_scrape + 1):
            proxies_to_iter = proxies_set

            for proxy in proxies_to_iter.copy():
                try:
                    headers = {
                        "authority": "www.etsy.com",
                        "sec-ch-ua": "^\\^Google",
                        "sec-ch-ua-mobile": "?0",
                        "upgrade-insecure-requests": "1",
                        "user-agent": str(ua.random),
                        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                        "sec-fetch-site": "same-origin",
                        "sec-fetch-mode": "navigate",
                        "sec-fetch-user": "?1",
                        "sec-fetch-dest": "document",
                        "referer": f"https://www.etsy.com/c/art-and-collectibles/{adj_keyword}?explicit=1&ship_to=US&ref=pagination&page={page_no-1}",
                        "accept-language": "en-US,en;q=0.9",
                    }
                    params = (
                        ("explicit", "1"),
                        ("ref", "pagination"),
                        ("page", f"{page_no}"),
                    )
                    page = requests.get(
                        f"https://www.etsy.com/c/art-and-collectibles/{adj_keyword}?explicit=1&ship_to=US&ref=pagination&page={page_no}",
                        headers=headers,
                        params=params,
                        proxies={"http": str(proxy), "https": str(proxy)},
                    )
                except requests.exceptions.RequestException as e:
                    proxies_to_iter.discard(proxy)
                    continue
                else:
                    break

            soup = BeautifulSoup(page.content, "html.parser")

            soup = replace_attr(page, "data-src", "src")

            for value in soup.find_all("div", class_=["js-merch-stash-check-listing"]):
                if value.find(class_="strike-through"):
                    value.unwrap()

                titles.extend([title.get("title") for title in value.find_all("a")])
                prices.extend(
                    [
                        float(price.get_text().replace(",", ""))
                        for price in value.find_all("span", class_="currency-value")
                    ]
                )
                item_urls.extend([link.get("href") for link in value.find_all("a")])
                img_urls.extend(
                    [
                        pic.img["src"]
                        for pic in value.find_all("div", class_="height-placeholder")
                    ]
                )

            time.sleep(np.random.uniform(0.4, 1.2))

        df_list.append(
            pd.DataFrame(
                {
                    "category": adj_keyword[:items_to_scrape],
                    "title": titles[:items_to_scrape],
                    "price": prices[:items_to_scrape],
                    "item_url": item_urls[:items_to_scrape],
                    "img_url": img_urls[:items_to_scrape],
                }
            )
        )

    return pd.concat(df_list, ignore_index=True)

In [ ]:
def push_to_heroku(df: pd.DataFrame) -> None:
    """
    Functions takes a dataframe as an argument, then connects to a Heroku Postgres database.
    In the database it creates two tables pushes the information into both of them.
    :param df: Takes a pandas dataframe.
    :return: None, data sits on Heroku Postgres database.
    """
    sql_connection = psycopg2.connect(
        database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT
    )
    cur = sql_connection.cursor()

    cur.execute(
        """
    CREATE TABLE IF NOT EXISTS categories (
        id serial PRIMARY KEY,
        category varchar(250)
    );

    CREATE TABLE IF NOT EXISTS items (
        id serial PRIMARY KEY,
        category_id int,
        title varchar(250),
        price float(2),
        item_url varchar(500),
        img_url varchar(500),
        FOREIGN KEY (category_id) REFERENCES categories(id)
    );

    """
    )

    unique_categories = df["category"].unique()

    for i in unique_categories:
        cur.execute(f"INSERT INTO categories (category) VALUES ('{i}');")

    sql_connection.commit()
    sql_connection.close()

    foreign_key_df = (
        pd.DataFrame(df["category"].unique(), columns=["category"])
        .reset_index()
        .rename(columns={"index": "category_id"})
    )
    foreign_key_df["category_id"] = np.arange(1, len(foreign_key_df) + 1)

    items_df = pd.merge(df, foreign_key_df, on="category").drop(columns="category")

    items_df.insert(0, "category_id", items_df.pop("category_id"))
    conn = create_engine(sqlalchemy_engine_url)
    items_df.to_sql(
        "items", conn, method="multi", if_exists="append", chunksize=10000, index=False
    )
    conn.dispose()

In [ ]:
def get_csv() -> None:
    """
    This functions takes a path as an argument as saves the etsy_data.csv file in provided local directory.
    :param path: Provide a path where to save a csv file.
    :return: None, csv file saved in the local directory.
    """
    sql_connection = psycopg2.connect(
        database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT
    )
    cur = sql_connection.cursor()

    s = "SELECT items.id, categories.category, items.title, items.price, items.item_url, items.img_url FROM items JOIN categories ON categories.id = items.category_id ORDER BY id ASC"

    SQL_for_file_output = "COPY ({0}) TO STDOUT WITH CSV HEADER".format(s)

    with open("etsy_data.csv", "w") as f_output:
        cur.copy_expert(SQL_for_file_output, f_output)
    files.download("etsy_data.csv")
    sql_connection.close()

Example:

In [ ]:
df = scrape_etsy(
    ["dolls-and-miniatures", "photography", "prints", "mixed-media-and-collage"], 100
)

In [ ]:
push_to_heroku()

In [ ]:
get_csv()